In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from glob import glob
from tqdm import tqdm_notebook as tqdm

In [94]:
cv2.rectangle?

In [92]:
full_fps = glob('/home/rob/HWR/datasets/zooniverse/*/Cropped*')
for full_fp in tqdm(full_fps):
    full_faked_fp = full_fp.replace('Cropped','FullFaked')
    if os.path.exists(full_faked_fp):
        continue
        
    cropped = cv2.imread(full_fp)
    label_fps = glob('/'.join(full_fp.split('/')[:-1]) + '/Label*')
    for label_fp in label_fps:
        label_row = df[df['orig_fp']==label_fp].iloc[0]
        faked_fps = glob(label_fp.replace('Label','Faked*'))
        if not len(faked_fps):
            print('wtf')
            continue
        
        faked_fp = np.random.choice(faked_fps)
        _type = faked_fp.split('_')[-1].split('0')[0]
        if _type == 'QR':
            continue
        faked = cv2.imread(faked_fp)
        angle = label_row['angle']
        index = label_row.name
        sx,sy,ex,ey = label_row['sy'],label_row['sx'],label_row['ey'],label_row['ex']
        avg_color = faked.mean(axis=(0,1)).astype('uint8').astype('float')
        faked_rotated = rotate_bound(faked,-angle,avg_color)
        ex = min(min(ex-sx,faked_rotated.shape[0]) + sx,cropped.shape[0])
        ey = min(min(ey-sy,faked_rotated.shape[1]) + sy,cropped.shape[1])
        for i in range(sx,ex):
            x = i-sx
            for j in range(sy,ey):
                y = j-sy
                if not (faked_rotated[x,y] == avg_color).all():
                    cropped[i,j] = faked_rotated[x,y] 
        
        df.loc[index,'type'] = _type
        #df.to_csv('box coords.csv')
        
    #cv2.imwrite(full_faked_fp, cropped)
    break

wtf
wtf
wtf
wtf
wtf
wtf


In [91]:
df.to_csv('box coords.csv')

In [83]:
ex-sx,ey-sy,faked_rotated.shape

(1440, 1624, (1440, 1624, 3))

In [86]:
i,j,cropped.shape

(2902, 53, (2902, 3327, 3))

In [68]:
def rotate_bound(image, angle, borderValue):
    # grab the dimensions of the image and then determine the
    # center
    (h, w) = image.shape[:2]
    (cX, cY) = (w / 2, h / 2)

    # grab the rotation matrix (applying the negative of the
    # angle to rotate clockwise), then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((cX, cY), angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])

    # compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))

    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY

    # perform the actual rotation and return the image
    return cv2.warpAffine(image, M, (nW, nH), borderValue=borderValue)

In [4]:
df = pd.read_csv('box coords.csv', index_col=0)
df['type'] = df['type'].map(lambda x: 'QR' if x == 'QR Code' else x)
df

,catalog_no,id,sx,sy,ex,ey,orig_fp,angle,type
0,221812,0,210,285,1261,713,/home/rob/HWR/datasets/zooniverse/221812/Label...,0.077486,NaN
1,221812,1,2289,626,2885,1124,/home/rob/HWR/datasets/zooniverse/221812/Label...,4.344374,NaN
2,221812,2,446,822,1948,1214,/home/rob/HWR/datasets/zooniverse/221812/Label...,2.547739,NaN
3,221812,3,2255,1159,3201,1656,/home/rob/HWR/datasets/zooniverse/221812/Label...,-5.173400,NaN
4,221812,4,962,1868,2552,2425,/home/rob/HWR/datasets/zooniverse/221812/Label...,0.105055,QR
5,221223,0,1712,1212,2669,1789,/home/rob/HWR/datasets/zooniverse/221223/Label...,-8.947473,NaN
6,221223,1,612,1233,1277,1644,/home/rob/HWR/datasets/zooniverse/221223/Label...,-2.882963,NaN
7,221223,2,677,2039,2251,2635,/home/rob/HWR/datasets/zooniverse/221223/Label...,-0.988999,QR
8,222460,0,1687,255,2662,675,/home/rob/HWR/datasets/zooniverse/222460/Label...,1.912178,NaN
9,222460,1,1875,843,3060,1357,/home/rob/HWR/datasets/zooniverse/222460/Label...,-2.965034,NaN
